In [2]:
import sys

from src.global_vars import BASE_DATA_DIR

sys.path.append("../")

import pandas as pd
from loguru import logger

import src.preprocessing.functions as preprocessing_functions

data_root_dir = f"{BASE_DATA_DIR}/sun_et_al_data/"
columns_to_keep = ["Sample", "Group", "Project", "Project_1"]
studies_to_remove = ["LiS_2021a", "LiS_2021b"]


def print_full_df(x):
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", None)
    pd.set_option("display.max_colwidth", None)
    display(x)
    pd.reset_option("display.max_rows")
    pd.reset_option("display.max_columns")
    pd.reset_option("display.width")
    pd.reset_option("display.float_format")
    pd.reset_option("display.max_colwidth")

# Pre-processing

### Preprocessing before splitting

In [4]:
# Get sample group data
sample_group = pd.read_table(f"{data_root_dir}/sample.group", sep="\t", header=0)
# Remove studies
logger.info(f"sample_group.shape before removal of studies: {sample_group.shape}")
sample_group = sample_group[~sample_group["Project_1"].isin(studies_to_remove)]
logger.info(f"sample_group.shape after removal of studies: {sample_group.shape}")

# Keep recommended columns
logger.info(f"sample_group.shape before column removal: {sample_group.shape}")
sample_group = sample_group[columns_to_keep]
logger.info(f"sample_group_useful.shape after column removal: {sample_group.shape}")
# Set index to Sample
sample_group = sample_group.set_index("Sample")
logger.info(f"sample_group_useful.shape after setting index: {sample_group.shape}")

# Get species profile data
mpa4_species_profile = pd.read_table(
    f"{data_root_dir}/mpa4_species.profile", sep="\t", header=0, index_col=0
)
# Remove species with no reads
mpa4_species_profile = mpa4_species_profile.loc[
    :, mpa4_species_profile.sum(axis=0) >= 1
]

## Remove repeated samples
logger.info(f"sample_group_useful.shape before removal: {sample_group.shape}")
sample_group = sample_group[~sample_group.index.duplicated(keep="first")]
logger.info(f"sample_group_useful.shape after removal: {sample_group.shape}")

# remove samples not in sample_group
logger.info(
    f"mpa4_species_profile.shape before filtering out samples without metadata: {mpa4_species_profile.shape}"
)
samples_to_keep = list(
    set(sample_group.index.tolist()) & set(mpa4_species_profile.columns.tolist())
)
mpa4_species_profile = mpa4_species_profile[samples_to_keep]
logger.info(
    f"mpa4_species_profile.shape after filtering out samples without metadata: {mpa4_species_profile.shape}"
)
mpa4_species_profile = mpa4_species_profile.T
logger.info(
    f"mpa4_species_profile.shape after transposing: {mpa4_species_profile.shape}"
)

# remove samples from sample_group that are not in mpa4_species_profile
logger.info(
    f"sample_group_useful.shape before filtering out samples not in mpa4_species_profile: {sample_group.shape}"
)
sample_group = sample_group.loc[samples_to_keep]
logger.info(
    f"sample_group_useful.shape after filtering out samples not in mpa4_species_profile: {sample_group.shape}"
)

# Normalize the data
logger.info(
    f"mpa4_species_profile summation before normalization: {mpa4_species_profile.sum(axis=1)}"
)
mpa4_species_profile = preprocessing_functions.total_sum_scaling(mpa4_species_profile)
logger.info(
    f"mpa4_species_profile summation after normalization: {mpa4_species_profile.sum(axis=1)}"
)


# normalize again

# transform

2025-02-04 14:35:56.742 | INFO     | __main__:<module>:4 - sample_group.shape before removal of studies: (6616, 21)
2025-02-04 14:35:56.746 | INFO     | __main__:<module>:6 - sample_group.shape after removal of studies: (6463, 21)
2025-02-04 14:35:56.747 | INFO     | __main__:<module>:9 - sample_group.shape before column removal: (6463, 21)
2025-02-04 14:35:56.751 | INFO     | __main__:<module>:11 - sample_group_useful.shape after column removal: (6463, 4)
2025-02-04 14:35:56.751 | INFO     | __main__:<module>:14 - sample_group_useful.shape after setting index: (6463, 3)
2025-02-04 14:36:10.876 | INFO     | __main__:<module>:26 - sample_group_useful.shape before removal: (6463, 3)
2025-02-04 14:36:10.884 | INFO     | __main__:<module>:28 - sample_group_useful.shape after removal: (6310, 3)
2025-02-04 14:36:10.884 | INFO     | __main__:<module>:31 - mpa4_species_profile.shape before filtering out samples without metadata: (2557, 8620)
2025-02-04 14:36:10.989 | INFO     | __main__:<modul

In [5]:
# prevalence and abundance filtering
# low abundance filtering per study
grouped_sample_group = sample_group.groupby("Project_1")
display(mpa4_species_profile)
for project, samples in grouped_sample_group.groups.items():
    logger.info(f"Project: {project}")
    rows_to_update = mpa4_species_profile.loc[samples]
    feature_prevalence = (rows_to_update > 0.0001).sum(axis=0) / rows_to_update.shape[0]
    low_abundance_features = feature_prevalence < 0.1

    df_masked = rows_to_update.mask(
        low_abundance_features | (rows_to_update <= 0.0001), 0
    )
    mpa4_species_profile.update(df_masked)

display(mpa4_species_profile)
display(mpa4_species_profile.sum(axis=1))
display(mpa4_species_profile.sum(axis=1).sort_values(ascending=True))

name,s__Phocaeicola_plebeius,s__Faecalibacterium_prausnitzii,s__Ruminococcus_sp_NSJ_71,s__Eubacterium_rectale,s__Bacteroides_uniformis,s__Clostridium_sp_AF15_49,s__Lachnospira_eligens,s__Roseburia_sp_AF02_12,s__Phocaeicola_vulgatus,s__Ruminococcus_bicirculans,...,s__Rodentibacter_myodis,s__Rhodococcus_hoagii,s__Pseudomonas_psychrophila,s__Pseudomonas_sp_DG56_2,s__Providencia_rustigianii,s__Pseudomonas_vranovensis,s__Pseudomonas_taetrolens,s__Pseudomonas_deceptionensis,s__Desulfobulbus_oralis,s__Bacteroides_reticulotermitis
SRR12005688,0.000000,0.107848,0.000000,0.216508,0.000000,0.040925,0.030266,0.0,0.000000,0.010115,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR8898458,0.000433,0.012850,0.000000,0.073000,0.000268,0.000000,0.000000,0.0,0.000099,0.001432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1398105,0.000000,0.004571,0.000000,0.000000,0.000339,0.000000,0.000000,0.0,0.042101,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1190566,0.000653,0.032060,0.000000,0.023550,0.001749,0.007555,0.013555,0.0,0.019018,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR413675,0.000000,0.029786,0.000000,0.000000,0.034749,0.000000,0.007905,0.0,0.000000,0.002487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERR2017810,0.000000,0.038234,0.000000,0.000000,0.010016,0.000000,0.000000,0.0,0.199284,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR2017427,0.000000,0.008467,0.000000,0.000000,0.059883,0.000000,0.000000,0.0,0.158798,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR8845378,0.000163,0.016779,0.010847,0.000105,0.088691,0.000000,0.003445,0.0,0.121519,0.001854,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yu_97,0.000000,0.002957,0.000000,0.005599,0.034018,0.003634,0.003383,0.0,0.041015,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


2025-02-04 14:36:39.878 | INFO     | __main__:<module>:6 - Project: ChenB_2020
2025-02-04 14:36:48.369 | INFO     | __main__:<module>:6 - Project: ChuY_2021
2025-02-04 14:36:55.668 | INFO     | __main__:<module>:6 - Project: HanL_2021
2025-02-04 14:37:03.399 | INFO     | __main__:<module>:6 - Project: HeQ_2017
2025-02-04 14:37:10.816 | INFO     | __main__:<module>:6 - Project: HuY_2019
2025-02-04 14:37:18.104 | INFO     | __main__:<module>:6 - Project: HuangR_2020
2025-02-04 14:37:25.393 | INFO     | __main__:<module>:6 - Project: JieZ_2017
2025-02-04 14:37:32.160 | INFO     | __main__:<module>:6 - Project: LiJ_2017
2025-02-04 14:37:39.400 | INFO     | __main__:<module>:6 - Project: LiR_2021
2025-02-04 14:37:46.363 | INFO     | __main__:<module>:6 - Project: LiuP_2021
2025-02-04 14:37:53.300 | INFO     | __main__:<module>:6 - Project: LiuR_2017
2025-02-04 14:38:00.176 | INFO     | __main__:<module>:6 - Project: LuW_2018
2025-02-04 14:38:06.989 | INFO     | __main__:<module>:6 - Project

name,s__Phocaeicola_plebeius,s__Faecalibacterium_prausnitzii,s__Ruminococcus_sp_NSJ_71,s__Eubacterium_rectale,s__Bacteroides_uniformis,s__Clostridium_sp_AF15_49,s__Lachnospira_eligens,s__Roseburia_sp_AF02_12,s__Phocaeicola_vulgatus,s__Ruminococcus_bicirculans,...,s__Rodentibacter_myodis,s__Rhodococcus_hoagii,s__Pseudomonas_psychrophila,s__Pseudomonas_sp_DG56_2,s__Providencia_rustigianii,s__Pseudomonas_vranovensis,s__Pseudomonas_taetrolens,s__Pseudomonas_deceptionensis,s__Desulfobulbus_oralis,s__Bacteroides_reticulotermitis
SRR12005688,0.000000,0.107848,0.000000,0.216508,0.000000,0.040925,0.030266,0.0,0.000000,0.010115,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR8898458,0.000433,0.012850,0.000000,0.073000,0.000268,0.000000,0.000000,0.0,0.000000,0.001432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1398105,0.000000,0.004571,0.000000,0.000000,0.000339,0.000000,0.000000,0.0,0.042101,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1190566,0.000653,0.032060,0.000000,0.023550,0.001749,0.007555,0.013555,0.0,0.019018,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR413675,0.000000,0.029786,0.000000,0.000000,0.034749,0.000000,0.007905,0.0,0.000000,0.002487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERR2017810,0.000000,0.038234,0.000000,0.000000,0.010016,0.000000,0.000000,0.0,0.199284,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR2017427,0.000000,0.008467,0.000000,0.000000,0.059883,0.000000,0.000000,0.0,0.158798,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR8845378,0.000163,0.016779,0.010847,0.000105,0.088691,0.000000,0.003445,0.0,0.121519,0.001854,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yu_97,0.000000,0.002957,0.000000,0.005599,0.034018,0.003634,0.003383,0.0,0.041015,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


SRR12005688    0.932501
SRR8898458     0.967762
ERR1398105     0.863804
ERR1190566     0.755357
SRR413675      0.869317
                 ...   
ERR2017810     0.768255
ERR2017427     0.975896
SRR8845378     0.749487
Yu_97          0.944817
ERR2855924     0.992831
Length: 6303, dtype: float64

SRR13077864    0.002350
ERR1620261     0.057627
SRR13077661    0.078934
SRR16124320    0.094270
ERR2017528     0.209203
                 ...   
ERR2868463     0.999717
ERR1398164     0.999763
SRR16124179    0.999781
SRR13077801    1.000000
ERR1620313     1.000000
Length: 6303, dtype: float64

In [11]:
# normalize and transform
logger.info(
    f"mpa4_species_profile summation before normalization: {mpa4_species_profile.sum(axis=1)}"
)
mpa4_species_profile = preprocessing_functions.total_sum_scaling(mpa4_species_profile)
logger.info(
    f"mpa4_species_profile summation after normalization: {mpa4_species_profile.sum(axis=1)}"
)

# Centered arcsine transform
logger.info(
    f"mpa4_species_profile summation before centered arcsine transform: {mpa4_species_profile.sum(axis=1)}"
)
mpa4_species_profile = preprocessing_functions.centered_arcsine_transform(
    mpa4_species_profile
)
logger.info(
    f"mpa4_species_profile summation after centered arcsine transform: {mpa4_species_profile.sum(axis=1)}"
)

2025-02-04 14:47:51.485 | INFO     | __main__:<module>:2 - mpa4_species_profile summation before normalization: SRR12005688    0.932501
SRR8898458     0.967762
ERR1398105     0.863804
ERR1190566     0.755357
SRR413675      0.869317
                 ...   
ERR2017810     0.768255
ERR2017427     0.975896
SRR8845378     0.749487
Yu_97          0.944817
ERR2855924     0.992831
Length: 6303, dtype: float64
2025-02-04 14:47:51.810 | INFO     | __main__:<module>:6 - mpa4_species_profile summation after normalization: SRR12005688    1.0
SRR8898458     1.0
ERR1398105     1.0
ERR1190566     1.0
SRR413675      1.0
              ... 
ERR2017810     1.0
ERR2017427     1.0
SRR8845378     1.0
Yu_97          1.0
ERR2855924     1.0
Length: 6303, dtype: float64
2025-02-04 14:47:51.942 | INFO     | __main__:<module>:11 - mpa4_species_profile summation before centered arcsine transform: SRR12005688    1.0
SRR8898458     1.0
ERR1398105     1.0
ERR1190566     1.0
SRR413675      1.0
              ... 
ERR201

In [12]:
# Save the data
mpa4_species_profile.to_csv(f"{data_root_dir}/mpa4_species_profile_preprocessed.csv")
sample_group.to_csv(f"{data_root_dir}/sample_group_preprocessed.csv")

---
# Testing dataloader

In [261]:
%load_ext autoreload
%autoreload 2

from torch import manual_seed
from torch.utils.data import DataLoader

manual_seed(0)

from src.data.sun_et_al import BinaryFewShotBatchSampler, MicrobiomeDataset

test_study = ["ChenB_2020"]
val_study = ["ChuY_2021"]

train_df = mpa4_species_profile.loc[
    sample_group.loc[~sample_group["Project_1"].isin(test_study + val_study)].index
]
assert train_df.shape[0] == 5892

test_df = mpa4_species_profile.loc[
    sample_group.loc[sample_group["Project_1"].isin(test_study)].index
]
assert test_df.shape[0] == 231

val_df = mpa4_species_profile.loc[
    sample_group.loc[sample_group["Project_1"].isin(val_study)].index
]
assert val_df.shape[0] == 180

meta_data = sample_group[["Group", "Project_1"]].rename(
    columns={"Project_1": "project", "Group": "label"}
)

train = MicrobiomeDataset(train_df, meta_data.loc[train_df.index])
test = MicrobiomeDataset(test_df, meta_data.loc[test_df.index])
val = MicrobiomeDataset(val_df, meta_data.loc[val_df.index])

sampler = BinaryFewShotBatchSampler(train, 50, True, True)
train_loader = DataLoader(train, batch_sampler=sampler)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [271]:
# samples, labels = next(iter(train_loader))
# print(samples.shape)
# print(labels.shape)
# print(samples)
# print(labels)


97
157
[np.int64(2862), np.int64(2924), np.int64(2938), np.int64(2893), np.int64(2761), np.int64(2820), np.int64(2953), np.int64(2772), np.int64(2968), np.int64(2816), np.int64(2877), np.int64(2935), np.int64(2768), np.int64(2850), np.int64(2773), np.int64(2769), np.int64(2866), np.int64(2872), np.int64(2837), np.int64(2770), np.int64(2867), np.int64(2843), np.int64(2815), np.int64(2864), np.int64(3009), np.int64(2915), np.int64(2808), np.int64(2925), np.int64(2887), np.int64(2765), np.int64(2838), np.int64(2950), np.int64(2817), np.int64(2895), np.int64(2776), np.int64(2759), np.int64(2771), np.int64(2780), np.int64(2818), np.int64(2868), np.int64(2806), np.int64(2763), np.int64(2918), np.int64(2898), np.int64(2910), np.int64(2891), np.int64(2847), np.int64(2827), np.int64(2873), np.int64(2849), np.int64(3011), np.int64(2783), np.int64(2845), np.int64(3004), np.int64(2781), np.int64(2870), np.int64(3006), np.int64(2931), np.int64(2963), np.int64(2941), np.int64(2932), np.int64(2946), 

: 

In [257]:
from src.preprocessing.functions import pandas_label_encoder

m = pandas_label_encoder(meta_data)
m = m.sort_index().reset_index(drop=True).groupby("project")[["label"]]
g = m.get_group(0)
g2 = g.groupby("label").groups

import numpy as np

np.array(list(g2.values())[0])

l = [
    1,
    2,
    3,
]
l.extend(np.array(list(g2.values())[0]))
print(l)

train.__getitem__(l[5])

(tensor([ 0.4558,  0.1662, -0.0019,  ..., -0.0019, -0.0019, -0.0019]),
 tensor(1.))

# Meta-Learning

In [ ]:
import sys

sys.path.append("../../")

import torch
from torch import nn

from src.models.maml import Model

def get_studies_desired_from_sun_et_al(data, metadata, studies):
    # Filter metadata to only include the studies of interest
    metadata = metadata[metadata["Project_1"].isin(studies)]
    # Filter data to only include samples that are in the metadata
    metadata = metadata.set_index("Sample")
    # Filter data to only include samples that are in the metadata
    labels = metadata["Group"]

    # Filter data to only include samples that are in the metadata
    data = data.loc[labels.index]

    return data, labels

def split_sun_et_al_data(data, metadata, test, val):
    """Split the data into train, test and validation sets

    Args:
        data (pd.DataFrame): The data to split
        test (list): The studies to use for testing
        val (list): The studies to use for validation

    Returns:
        tuple: train, test, val dataframes
    """
    if not isinstance(test, list):
        test = [test]
    if not isinstance(val, list):
        val = [val]

    test_data


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    sun_et_al_abundance = pd.read_csv(f"{data_root_dir}/mpa4_species_profile_preprocessed.csv")
    sun_et_al_metadata = pd.read_csv(f"{data_root_dir}/sample_group_preprocessed.csv")



    # Simple model to test
    model = Model(
        nodes_per_layer=[n_features, 64, 128, 64, 1],
        activations_per_layer=[nn.ReLU(), nn.ReLU(), nn.ReLU(), nn.sigmoid()],
    ).to(device)